Dataloaders

In [20]:
import os
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


def create_dataloaders(train_dir, val_dir, test_dir, batch_size=32):
    """
    Create DataLoaders for train, validation, and test datasets.

    Args:
        train_dir (str): Path to the training directory with augmented images.
        val_dir (str): Path to the validation directory.
        test_dir (str): Path to the test directory.
        batch_size (int): Batch size for DataLoader.

    Returns:
        train_loader, val_loader, test_loader, class_names
    """
    #transforms
    train_transforms = transforms.Compose([
        transforms.Resize((128, 128)),   
        transforms.ToTensor(),          
        transforms.Normalize([0.5], [0.5])  # normalize images to mean=0.5, std=0.5
    ])

    test_transforms = transforms.Compose([
        transforms.Resize((128, 128)),  
        transforms.ToTensor(),          
        transforms.Normalize([0.5], [0.5])  
    ])

    # load datasets
    train_dataset = ImageFolder(root=train_dir, transform=train_transforms)
    val_dataset = ImageFolder(root=val_dir, transform=test_transforms)
    test_dataset = ImageFolder(root=test_dir, transform=test_transforms)

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, train_dataset.classes

Training / Validation

In [21]:
import torch
import tqdm
from torch.utils.tensorboard import SummaryWriter
from torchmetrics import F1Score

# Configuration
save_model_path = "checkpoints/"
pth_name = "saved_model.pth"
os.makedirs(save_model_path, exist_ok=True)

def val(model, val_loader, loss_function, writer, epoch, device):
    f1 = F1Score(num_classes=len(val_loader.dataset.classes),average='weighted', task='multiclass')
    val_iterator = enumerate(val_loader)
    f1_list, f1t_list = [], []
    total_loss = 0
    total_correct = 0
    total_samples = 0

    model.eval()
    tq = tqdm.tqdm(total=len(val_loader), desc="Validation")
    with torch.no_grad():
        for _, (images, labels) in val_iterator:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            predictions = model(images)
            loss = loss_function(predictions, labels.long())  # Change labels to .long()
            total_loss += loss.item()

            # Compute predictions
            #predictions = predictions.softmax(dim=1)
            predictions = torch.argmax(predictions, dim=1)
            f1_list.extend(predictions.cpu().numpy())
            f1t_list.extend(labels.cpu().numpy())
            # Calculate accuracy
            #_, predicted = torch.max(predictions, 1)
            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)


            tq.update(1)

    tq.close()

    # Calculate F1 Score
    f1_score_value = f1(torch.tensor(f1_list), torch.tensor(f1t_list))
    val_accuracy = total_correct / total_samples

    writer.add_scalar("Validation F1", f1_score_value, epoch)
    writer.add_scalar("Validation Loss", total_loss / len(val_loader), epoch)
    writer.add_scalar("Validation Accuracy", val_accuracy, epoch)

    #print(f'list_pred {f1_list}')
    #print(f'list_pred {f1t_list}')
    print(f"Validation Loss: {total_loss / len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {f1_score_value:.4f}")



def train(model, train_loader, val_loader, optimizer, loss_fn, n_epochs, device):
    """
    Train the model on the training dataset.
    Logs loss and saves the model at each epoch.

    Args:
        model: PyTorch model.
        train_loader: DataLoader for training data.
        val_loader: DataLoader for validation data.
        optimizer: Optimizer for training.
        loss_fn: Loss function.
        n_epochs: Number of epochs.
        device: Device for computation (CPU/GPU).
    """
    writer = SummaryWriter()

    model.to(device)
    for epoch in range(n_epochs):
        model.train()  
        running_loss = 0.0
        total_correct = 0
        total_samples = 0
        f1 = F1Score(num_classes=len(train_loader.dataset.classes),average='weighted', task='multiclass').to(device)  # Move F1 metric to device

        tq = tqdm.tqdm(total=len(train_loader), desc=f"Epoch {epoch + 1}/{n_epochs}")
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Zero the gradient
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = loss_fn(outputs, labels)

            # Backward pass and optimizer step
            loss.backward()
            optimizer.step()

            # Update running loss
            running_loss += loss.item()

            # Compute predictions
            predictions = torch.argmax(outputs, dim=1)

            # Update F1 score
            f1.update(predictions, labels)
            
            # Accuracy calculation
            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)

            tq.set_postfix(loss=loss.item())
            tq.update(1)

        tq.close()

        # Compute F1 score at the end of the epoch
        f1_score_value = f1.compute()

        # Calculate training accuracy
        train_accuracy = total_correct / total_samples
        epoch_loss = running_loss / len(train_loader)

        # Log training metrics
        writer.add_scalar("Training Loss", epoch_loss, epoch)
        writer.add_scalar("Training Accuracy", train_accuracy, epoch)
        writer.add_scalar("Training F1", f1_score_value, epoch)

        # Print training metrics
        print(f"Epoch [{epoch + 1}/{n_epochs}], Training Loss: {epoch_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Training F1: {f1_score_value:.4f}")

        # Validate the model
        val(model, val_loader, loss_fn, writer, epoch, device)

        # Save the model checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        torch.save(checkpoint, os.path.join(save_model_path, pth_name))
        print(f"Model saved at {save_model_path}{pth_name}")

Test

In [22]:
import torch
from sklearn.metrics import accuracy_score, f1_score
import tqdm  # Correct import for tqdm
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

def test(model, test_data_path, device, batch_size=32, image_size=(128, 128), mean=[0.5], std=[0.5]):
    """
    Evaluates a trained model on a test dataset.

    Parameters:
    - model (torch.nn.Module): The model to evaluate.
    - test_data_path (str): Path to the test dataset.
    - device (torch.device): The device to run the evaluation on (CPU or CUDA).
    - batch_size (int, optional): The batch size for the DataLoader. Default is 32.
    - image_size (tuple, optional): The target image size for the input images. Default is (128, 128).
    - mean (list, optional): The mean for normalization. Default is [0.5].
    - std (list, optional): The std for normalization. Default is [0.5].

    Returns:
    - None: Prints the accuracy and F1 score of the model on the test set.
    """

    # Load test dataset with transformations
    test_transforms = transforms.Compose([
        transforms.Resize(image_size),  # Resize to match input size for the model
        transforms.ToTensor(),          # Convert PIL images to tensors
        transforms.Normalize(mean=mean, std=std)  # Normalize (same mean/std as training)
    ])

    test_dataset = ImageFolder(root=test_data_path, transform=test_transforms)
    class_names = test_dataset.classes  # Retrieve class names
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Set the model to evaluation mode
    model.eval()

    # Initialize variables to store true labels and predicted labels
    all_true_labels = []
    all_pred_labels = []

    # Evaluate the model
    with torch.no_grad():  # No need to compute gradients during inference
        for inputs, labels in tqdm.tqdm(test_loader, desc="Evaluating"):  # Use tqdm from the module
            # Move the inputs and labels to the device (GPU or CPU)
            inputs, labels = inputs.to(device), labels.to(device)

            # Get model predictions
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)  # Get the predicted class (index of max logit)

            # Append true and predicted labels to the lists
            all_true_labels.extend(labels.cpu().numpy())
            all_pred_labels.extend(preds.cpu().numpy())

    # Compute accuracy and F1 score
    accuracy = accuracy_score(all_true_labels, all_pred_labels)
    f1 = f1_score(all_true_labels, all_pred_labels, average='macro')  # Weighted F1 score

    # Print results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score (Macro): {f1:.4f}")


In [ ]:
train_dir = 'dataset/train'
val_dir = 'dataset/validation' 
test_dir = 'dataset/test'         
log_dir = "outputs/logs"        
model_dir = "outputs/models"    # Model save path
batch_size = 32
num_epochs = 2
device = torch.device("cuda")

initializing dataloaders

In [24]:
train_loader, val_loader, test_loader, class_names = create_dataloaders(
    train_dir=train_dir,
    val_dir=val_dir,
    test_dir=test_dir,
    batch_size=batch_size
)


<h1>Resnet18

<h2>Resnet18 pretrained

In [25]:
import torch.nn as nn
import torchvision.models as models

def get_resnet18_pretrained(num_classes, transfer_learning=True):
    model = models.resnet18(pretrained=transfer_learning)
    if transfer_learning:
        for param in model.parameters():
            param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

with sgd

In [34]:
# ResNet18 Training
print("Training ResNet18...")

# Initialize the ResNet18 model with the number of classes
resnet18_pretrained_sgd = get_resnet18_pretrained(num_classes=len(class_names)).to(device)

# Define optimizers
optimizer_sgd = torch.optim.SGD(resnet18_pretrained_sgd.parameters(), lr=0.0005, momentum=0.9)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=resnet18_pretrained_sgd,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_sgd,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(resnet18_pretrained_sgd.state_dict(), os.path.join(model_dir, "resnet18_pretrained_sgd.pth"))
print(f"ResNet18 model saved to {os.path.join(model_dir, 'resnet18_pretrained_sgd.pth')}")

# Test the model
print("\nTest score:")
test(resnet18_pretrained_sgd, test_data_path=test_dir, device=device)

Training ResNet18...


c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/2: 100%|██████████| 90/90 [00:10<00:00,  8.30it/s, loss=0.745]


Epoch [1/2], Training Loss: 0.6355, Training Accuracy: 0.6697, Training F1: 0.6697


Validation: 100%|██████████| 6/6 [00:00<00:00, 10.12it/s]


Validation Loss: 0.5304, Validation Accuracy: 0.7444, Validation F1 Score: 0.7398
Model saved at checkpoints/saved_model.pth


...

Epoch [2/2], Training Loss: 0.4865, Training Accuracy: 0.7629, Training F1: 0.7629


Validation: 100%|██████████| 6/6 [00:00<00:00, 11.50it/s]


Validation Loss: 0.5018, Validation Accuracy: 0.7611, Validation F1 Score: 0.7564
Model saved at checkpoints/saved_model.pth
ResNet18 model saved to outputs/models\resnet18_pretrained_sgd.pth

Test score:


Evaluating: 100%|██████████| 7/7 [00:00<00:00, 10.65it/s]

Accuracy: 0.7321
F1 Score (Macro): 0.7304


with Adam

In [35]:
resnet18_pretrained_adam = get_resnet18_pretrained(num_classes=len(class_names)).to(device)
optimizer_adam = torch.optim.Adam(resnet18_pretrained_adam.parameters(), lr=0.0001)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=resnet18_pretrained_adam,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_adam,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(resnet18_pretrained_adam.state_dict(), os.path.join(model_dir, "resnet18_pretrained_adam.pth"))
print(f"ResNet18 model saved to {os.path.join(model_dir, 'resnet18_pretrained_adam.pth')}")

# Test the model
print("\nTest score:")
test(resnet18_pretrained_adam, test_data_path=test_dir, device=device)

c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/2: 100%|██████████| 90/90 [00:10<00:00,  9.00it/s, loss=0.619]


Epoch [1/2], Training Loss: 0.7232, Training Accuracy: 0.5548, Training F1: 0.5545


Validation: 100%|██████████| 6/6 [00:00<00:00, 13.73it/s]


Validation Loss: 0.6957, Validation Accuracy: 0.6444, Validation F1 Score: 0.6369
Model saved at checkpoints/saved_model.pth


Epoch 2/2: 100%|██████████| 90/90 [00:07<00:00, 12.21it/s, loss=0.52] 


Epoch [2/2], Training Loss: 0.6502, Training Accuracy: 0.6278, Training F1: 0.6278


Validation: 100%|██████████| 6/6 [00:00<00:00, 10.84it/s]


Validation Loss: 0.6645, Validation Accuracy: 0.6722, Validation F1 Score: 0.6524
Model saved at checkpoints/saved_model.pth
ResNet18 model saved to outputs/models\resnet18_pretrained_adam.pth

Test score:


Evaluating: 100%|██████████| 7/7 [00:00<00:00,  8.82it/s]

Accuracy: 0.6071
F1 Score (Macro): 0.5789


<h2>Resnet18 not-pretrained

In [30]:
def get_resnet18_np(num_classes, transfer_learning=False):
    model = models.resnet18(pretrained=False)  # Do not load pre-trained weights
    '''if transfer_learning:
        for param in model.parameters():
            param.requires_grad = False  # Freeze all layers if transfer learning '''
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

with sgd

In [41]:
# ResNet18 Training
print("Training ResNet18...")

# Initialize the ResNet18 model with the number of classes
resnet18_sgd = get_resnet18_np(num_classes=len(class_names)).to(device)

# Define optimizers
optimizer_sgd = torch.optim.SGD(resnet18_sgd.parameters(), lr=0.0005, momentum=0.9)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=resnet18_sgd,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_sgd,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(resnet18_sgd.state_dict(), os.path.join(model_dir, "resnet18_sgd.pth"))
print(f"ResNet18 model saved to {os.path.join(model_dir, 'resnet18_sgd.pth')}")

# Test the model
print("\nTest score:")
test(resnet18_sgd, test_data_path=test_dir, device=device)

Training ResNet18...


c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/2: 100%|██████████| 90/90 [00:09<00:00,  9.01it/s, loss=0.343]


Epoch [1/2], Training Loss: 0.5869, Training Accuracy: 0.6858, Training F1: 0.6858


Validation: 100%|██████████| 6/6 [00:00<00:00, 11.79it/s]


Validation Loss: 0.5309, Validation Accuracy: 0.7889, Validation F1 Score: 0.7868
Model saved at checkpoints/saved_model.pth


...

Epoch [2/2], Training Loss: 0.5004, Training Accuracy: 0.7542, Training F1: 0.7541


Validation: 100%|██████████| 6/6 [00:00<00:00, 13.74it/s]


Validation Loss: 0.5192, Validation Accuracy: 0.7722, Validation F1 Score: 0.7719
Model saved at checkpoints/saved_model.pth
ResNet18 model saved to outputs/models\resnet18_sgd.pth

Test score:


Evaluating: 100%|██████████| 7/7 [00:00<00:00, 12.56it/s]

Accuracy: 0.7143
F1 Score (Macro): 0.7110


with adam

In [42]:
resnet18_adam = get_resnet18_np(num_classes=len(class_names)).to(device)
optimizer_adam = torch.optim.Adam(resnet18_adam.parameters(), lr=0.0001)

criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=resnet18_adam,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_adam,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(resnet18_adam.state_dict(), os.path.join(model_dir, "resnet18_adam.pth"))
print(f"ResNet18 model saved to {os.path.join(model_dir, 'resnet18_adam.pth')}")

# Test the model
print("\nTest score:")
test(resnet18_adam, test_data_path=test_dir, device=device)

c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/2: 100%|██████████| 90/90 [00:10<00:00,  8.96it/s, loss=0.0642]


Epoch [1/2], Training Loss: 0.3491, Training Accuracy: 0.8425, Training F1: 0.8424


Validation: 100%|██████████| 6/6 [00:00<00:00, 13.78it/s]


Validation Loss: 0.6430, Validation Accuracy: 0.7500, Validation F1 Score: 0.7494
Model saved at checkpoints/saved_model.pth


Epoch 2/2: 100%|██████████| 90/90 [00:08<00:00, 11.10it/s, loss=0.132]  


Epoch [2/2], Training Loss: 0.0616, Training Accuracy: 0.9797, Training F1: 0.9797


Validation: 100%|██████████| 6/6 [00:00<00:00, 13.56it/s]


Validation Loss: 0.8122, Validation Accuracy: 0.7444, Validation F1 Score: 0.7419
Model saved at checkpoints/saved_model.pth
ResNet18 model saved to outputs/models\resnet18_adam.pth

Test score:


Evaluating: 100%|██████████| 7/7 [00:00<00:00, 13.28it/s]

Accuracy: 0.7366
F1 Score (Macro): 0.7357


<h1>VGG16

<h2> Pretrained

In [36]:
def get_vgg16_pretrained(num_classes, transfer_learning=True):
    model = models.vgg16(pretrained=transfer_learning)
    if transfer_learning:
        for param in model.features.parameters():
            param.requires_grad = False
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    return model    

with sgd

In [ ]:
# VGG16 Training
print("Training VGG16...")

vgg16_pretrained_sgd = get_vgg16_pretrained(num_classes=len(class_names)).to(device)

optimizer_sgd = torch.optim.SGD(vgg16_pretrained_sgd.parameters(), lr=0.0005, momentum=0.9)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=vgg16_pretrained_sgd,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_sgd,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(vgg16_pretrained_sgd.state_dict(), os.path.join(model_dir, "vgg16_pretrained_sgd.pth"))
print(f"VGG16 model saved to {os.path.join(model_dir, 'vgg16_pretrained_sgd.pth')}")

# Test the model
print("\nTest score:")
test(vgg16_pretrained_sgd, test_data_path=test_dir, device=device)


Training VGG16...


c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/2: 100%|██████████| 90/90 [00:26<00:00,  3.41it/s, loss=0.684]


Epoch [1/2], Training Loss: 0.6878, Training Accuracy: 0.5590, Training F1: 0.5481


Validation: 100%|██████████| 6/6 [00:00<00:00,  8.26it/s]


Validation Loss: 0.6738, Validation Accuracy: 0.6556, Validation F1 Score: 0.6156
Model saved at checkpoints/saved_model.pth


...

Epoch [2/2], Training Loss: 0.6256, Training Accuracy: 0.7022, Training F1: 0.6936


Validation: 100%|██████████| 6/6 [00:00<00:00,  7.26it/s]


Validation Loss: 0.5634, Validation Accuracy: 0.7333, Validation F1 Score: 0.7333
Model saved at checkpoints/saved_model.pth
VGG16 model saved to outputs/models\vgg16_pretrained_sgd.pth

Test score:


Evaluating: 100%|██████████| 7/7 [00:01<00:00,  6.31it/s]

Accuracy: 0.6964
F1 Score (Macro): 0.6958


with Adam

In [ ]:
print("Training VGG16...")

vgg16_pretrained_adam = get_vgg16_pretrained(num_classes=len(class_names)).to(device)

optimizer_adam = torch.optim.Adam(vgg16_pretrained_adam.parameters(), lr=0.00001)

criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=vgg16_pretrained_adam,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_adam,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(vgg16_pretrained_adam.state_dict(), os.path.join(model_dir, "vgg16_pretrained_adam.pth"))
print(f"VGG16 model saved to {os.path.join(model_dir, 'vgg16_pretrained_adam.pth')}")

# Test the model
print("\nTest score:")
test(vgg16_pretrained_adam, test_data_path=test_dir, device=device)


Training VGG16...


c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/3: 100%|██████████| 90/90 [00:15<00:00,  5.83it/s, loss=0.685]


Epoch [1/3], Training Loss: 0.6083, Training Accuracy: 0.6676, Training F1: 0.6674


Validation: 100%|██████████| 6/6 [00:00<00:00,  7.52it/s]


Validation Loss: 0.4521, Validation Accuracy: 0.8000, Validation F1 Score: 0.7964
Model saved at checkpoints/saved_model.pth


...

Epoch [3/3], Training Loss: 0.3252, Training Accuracy: 0.8680, Training F1: 0.8680


Validation: 100%|██████████| 6/6 [00:00<00:00,  7.84it/s]


Validation Loss: 0.3026, Validation Accuracy: 0.8778, Validation F1 Score: 0.8774
Model saved at checkpoints/saved_model.pth
VGG16 model saved to outputs/models\vgg16_pretrained_adam.pth

Test score:


Evaluating: 100%|██████████| 7/7 [00:01<00:00,  6.11it/s]

Accuracy: 0.8348
F1 Score (Macro): 0.8348


<h2>Not-pretrained

In [47]:
def get_vgg16_np(num_classes, transfer_learning=False):
    model = models.vgg16(pretrained=False)  # Do not load pre-trained weights
    ''''if transfer_learning:
        for param in model.features.parameters():
            param.requires_grad = False  # Freeze all layers of 'features' if transfer learning'''
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    return model

with sgd

In [48]:
print("Training VGG16 (without pretrained weights)...")

# Initialize the VGG16 model without pretrained weights
vgg16_sgd_np = get_vgg16_np(num_classes=len(class_names)).to(device)

# Define optimizers
optimizer_sgd = torch.optim.SGD(vgg16_sgd_np.parameters(), lr=0.0005, momentum=0.9)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=vgg16_sgd_np,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_sgd,
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(vgg16_sgd_np.state_dict(), os.path.join(model_dir, "vgg16_sgd_np.pth"))
print(f"VGG16 model saved to {os.path.join(model_dir, 'vgg16_sgd_np.pth')}")

# Test the model
print("\nTest score:")
test(vgg16_sgd_np, test_data_path=test_dir, device=device)


Training VGG16 (without pretrained weights)...


c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/2: 100%|██████████| 90/90 [00:26<00:00,  3.44it/s, loss=0.667]


Epoch [1/2], Training Loss: 0.6837, Training Accuracy: 0.5817, Training F1: 0.5806


Validation: 100%|██████████| 6/6 [00:00<00:00,  6.85it/s]


Validation Loss: 0.6662, Validation Accuracy: 0.6222, Validation F1 Score: 0.6065
Model saved at checkpoints/saved_model.pth


...

Epoch [2/2], Training Loss: 0.5955, Training Accuracy: 0.7119, Training F1: 0.7119


Validation: 100%|██████████| 6/6 [00:00<00:00,  8.61it/s]


Validation Loss: 0.5414, Validation Accuracy: 0.7667, Validation F1 Score: 0.7631
Model saved at checkpoints/saved_model.pth
VGG16 model saved to outputs/models\vgg16_sgd_np.pth

Test score:


Evaluating: 100%|██████████| 7/7 [00:01<00:00,  6.19it/s]

Accuracy: 0.7009
F1 Score (Macro): 0.6915


with adam

In [49]:
print("Training VGG16 (without pretrained weights)...")

# Initialize the VGG16 model without pretrained weights
vgg16_adam_np = get_vgg16_np(num_classes=len(class_names)).to(device)

optimizer_adam = torch.optim.Adam(vgg16_adam_np.parameters(), lr=0.0005)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(
    model=vgg16_adam_np,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer_adam,  
    loss_fn=criterion,
    n_epochs=num_epochs,
    device=device
)

# Saving the trained model
torch.save(vgg16_adam_np.state_dict(), os.path.join(model_dir, "vgg16_adam_np.pth"))
print(f"VGG16 model saved to {os.path.join(model_dir, 'vgg16_adam_np.pth')}")

# Test the model
print("\nTest score:")
test(vgg16_adam_np, test_data_path=test_dir, device=device)


Training VGG16 (without pretrained weights)...


c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\Desktop\AI_Health_Project\stroke\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/2: 100%|██████████| 90/90 [02:07<00:00,  1.42s/it, loss=0.685]


Epoch [1/2], Training Loss: 0.8579, Training Accuracy: 0.5440, Training F1: 0.5414


Validation: 100%|██████████| 6/6 [00:01<00:00,  5.37it/s]


Validation Loss: 0.6959, Validation Accuracy: 0.5000, Validation F1 Score: 0.3333
Model saved at checkpoints/saved_model.pth


...

Epoch [2/2], Training Loss: 0.6366, Training Accuracy: 0.6386, Training F1: 0.6370


Validation: 100%|██████████| 6/6 [00:01<00:00,  5.17it/s]


Validation Loss: 0.6168, Validation Accuracy: 0.6889, Validation F1 Score: 0.6864
Model saved at checkpoints/saved_model.pth
VGG16 model saved to outputs/models\vgg16_adam_np.pth

Test score:


Evaluating: 100%|██████████| 7/7 [00:01<00:00,  4.98it/s]

Accuracy: 0.7009
F1 Score (Macro): 0.7004
